## Nuevas variables **Diferencias**

Con el objetivo de mejorar los resultados obtenidos en los modelos de P2, se ha decidido crear nuevas variables a partir de las variables de las medias ponderadas. Las variables que se han creado son la diferencia entre las estadísticas de cada peleador, por ejemplo: de tener KD_A y KD_B, pasamos a tener KD_DIFF.  

En este proceso, observamos que un experto en el área no necesariamente se enfocaría en las estadísticas de cada peleador de manera individual. En lugar de eso, podría ser más útil analizar la diferencia entre las estadísticas de los dos peleadores. Es decir, en lugar de considerar los datos de cada peleador por separado, se podría prestar más atención a cómo varían esas estadísticas de un peleador a otro.  

Por esta razón, calculamos la diferencia en las estadísticas entre los peleadores y luego probamos varios modelos de predicción para ver si este enfoque ofrecía mejores resultados que trabajar solo con las estadísticas individuales de cada luchador.

* [Carga de los datos](#Carga-de-los-datos)  
* [Crear nuevas variables](#Crear-nuevas-variables)  
* [Partición de los datos](#particion-de-los-datos)
* [Guardar datos](#guardar-datos)

In [2]:
import pandas as pd
import os

### Carga de los datos

In [3]:
ruta = os.path.join(os.getcwd(), "..", "..", "..","data", "processed", "peleas_ponderadas.parquet")
df = pd.read_parquet(ruta)

### Filtrado del *dataframe*
Eliminamos datos anteriores al 2010 debido a que siempre ganaba el Peleador A y los datos generaban muchos fallos al entrenar el modelo generando grandes desbalances

In [4]:
df = df[df["DATE"].dt.year >= 2010]

### Crear nuevas variables

Se separan las colunmas de cada peleador y se calcula la diferencia. También se elimina del *dataframe* las columnas que no sean diferencia.

In [5]:
# Lista de columnas que tienen contraparte en A y B
columnas_a = ['KD_A', 'SIG_STR_A', 'TD_PORC_A', 'SUB_ATT_A', 'REV_A', 'CTRL_A', 
              'TOTAL_STR_A_x', 'TOTAL_STR_A_y', 'TD_A_x', 'TD_A_y', 
              'STR_HEAD_A_x', 'STR_HEAD_A_y', 'STR_BODY_A_x', 'STR_BODY_A_y', 
              'STR_LEG_A_x', 'STR_LEG_A_y', 'STR_DISTANCE_A_x', 'STR_DISTANCE_A_y', 
              'STR_CLINCH_A_x', 'STR_CLINCH_A_y', 'STR_GROUND_A_x', 'STR_GROUND_A_y', 
              'STRIKER_A', 'GRAPPLER_A', 'Record_A', 'Peleas_A', 'Puntos_A', 
              'Racha_A', 'Victorias_KO_A', 'Victorias_Sub_A', 'Victorias_Decision_A', 
              'Derrotas_KO_A', 'Derrotas_Sub_A', 'Derrotas_Decision_A']

columnas_b = ['KD_B', 'SIG_STR_B', 'TD_PORC_B', 'SUB_ATT_B', 'REV_B', 'CTRL_B', 
              'TOTAL_STR_B_x', 'TOTAL_STR_B_y', 'TD_B_x', 'TD_B_y', 
              'STR_HEAD_B_x', 'STR_HEAD_B_y', 'STR_BODY_B_x', 'STR_BODY_B_y', 
              'STR_LEG_B_x', 'STR_LEG_B_y', 'STR_DISTANCE_B_x', 'STR_DISTANCE_B_y', 
              'STR_CLINCH_B_x', 'STR_CLINCH_B_y', 'STR_GROUND_B_x', 'STR_GROUND_B_y', 
              'STRIKER_B', 'GRAPPLER_B', 'Record_B', 'Peleas_B', 'Puntos_B', 
              'Racha_B', 'Victorias_KO_B', 'Victorias_Sub_B', 'Victorias_Decision_B', 
              'Derrotas_KO_B', 'Derrotas_Sub_B', 'Derrotas_Decision_B']

# Calcular las diferencias
for col_a, col_b in zip(columnas_a, columnas_b):
    if col_a in df.columns and col_b in df.columns:
        df[f'{col_a[:-2]}_DIFF'] = df[col_a] - df[col_b]

# Eliminar las columnas originales
df.drop(columns=columnas_a + columnas_b, inplace=True)

# Mostrar el DataFrame resultante
df.head()

,DATE,Peleador_A,Peleador_B,WINNER,KD_DIFF,SIG_STR_DIFF,TD_DIFF,SUB_ATT_DIFF,REV_DIFF,CTRL_DIFF,...,Record_DIFF,Peleas_DIFF,Puntos_DIFF,Racha_DIFF,Victorias_KO_DIFF,Victorias_Sub_DIFF,Victorias_Decision_DIFF,Derrotas_KO_DIFF,Derrotas_Sub_DIFF,Derrotas_Decision_DIFF
0,2011-01-22,Melvin Guillard,Evan Dunham,False,0.24,0.1160,-0.475248,-3.16,0.80,-185.76,...,25.0,7,10.302760,3,3.6,-1.0,0.76,0.00,4.0,-0.36
1,2011-02-26,George Sotiropoulos,Dennis Siver,True,-0.12,-0.0364,0.081182,1.56,0.00,228.72,...,1.0,-3,3.779920,5,-2.0,2.0,1.00,-1.00,-1.0,-2.00
2,2011-03-19,Ricardo Almeida,Mike Pyle,True,-0.24,-0.0076,0.255163,-1.08,-0.24,238.92,...,-3.0,5,9.316743,-1,0.0,1.4,2.00,0.00,-0.4,1.00
3,2011-03-19,Mirko Filipovic,Brendan Schaub,True,-0.76,-0.1868,0.000000,0.88,0.00,143.08,...,-8.0,4,-8.023557,-3,1.4,0.6,-0.36,1.36,0.0,1.00
4,2011-03-19,Dan Miller,Nate Marquardt,True,0.00,-0.0880,0.093856,-0.92,-0.40,188.52,...,-14.0,-5,6.343023,2,-3.6,0.6,-1.64,-1.00,0.0,0.64


In [ ]:
ruta_dif =  os.path.join("..","..","..", "data", "P2_dif","completo_dif.parquet")
df.to_parquet(ruta_dif, index=False)

### Particion de los datos
Se dividen los datos en train y test, respetando el orden temporal

In [6]:
# Ordenar por fecha (si no está ordenado ya)
df = df.sort_values(by="DATE")

# Definir los tamaños de cada subconjunto
train_size = 0.8  # 80%
test_size = 0.2    # 20%

# Calcular índice de corte
n = len(df)
train_end = int(n * train_size)

# Dividir el DataFrame
df_train = df.iloc[:train_end]
df_test = df.iloc[train_end:]

# Obtener las fechas de corte
train_end_date = df_train["DATE"].max()
test_end_date = df_test["DATE"].max()

print(f"Última fecha en train: {train_end_date}")
print(f"Última fecha en test: {test_end_date}")

# Mostrar tamaños
print(f"Train: {len(df_train)}, Test: {len(df_test)}")


Última fecha en train: 2023-02-04 00:00:00
Última fecha en test: 2025-02-15 00:00:00
Train: 2372, Test: 594


Se verifica la distribución de las clases tras la división. Como se observa están muy poco desbalanceadas

In [7]:
# Verificar tamaños y balance
def check_distribution(data, name):
    total = len(data)
    class_dist = data["WINNER"].value_counts(normalize=True).round(2)
    print(f"{name}: {total} muestras ({total/len(df):.0%}) | Distribución: {class_dist.to_dict()}")

check_distribution(df_train, "Entrenamiento")
check_distribution(df_test, "Prueba")

Entrenamiento: 2372 muestras (80%) | Distribución: {False: 0.56, True: 0.44}
Prueba: 594 muestras (20%) | Distribución: {False: 0.55, True: 0.45}


### Guardar datos
Se guardan los *dataframes* en archivos *parquet*

In [8]:
base_dir = os.path.join(os.getcwd(), "..", "..", "..","data")
ruta_partition = os.path.join(base_dir, "P2_dif")

os.makedirs(ruta_partition, exist_ok=True)

df_train.to_parquet(os.path.join(ruta_partition, "train.parquet"), index = False)
df_test.to_parquet(os.path.join(ruta_partition, "test.parquet"), index = False)